# Results of white-box MIAs on the ResNet18 dataset

The models are trained using a ResNet18 architecture. 

The specification of this architecture is given in ```../configs/tiny-imagenet-200/resnet18.ini```.

We load results for MIAs trained using different features (specified below as ```activations```, ```gradients```, or ```activations,gradients```), extracted from different layers (e.g., ```fc1``` which stands for the first fully connected layer). The features can be extracted from different types of shadow models, indicated in the ```attacker_accessess``` list below. We hope the naming to be self-explanatory. We refer you to ```vgg_shadow_modelling_attack.py``` for more details on the different types of shadow models. 

In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import scipy.stats

In [2]:
attacker_accesses = ["aa-target_dataset",
    "aa-shadow_dataset_model_init",
    "aa-shadow_dataset",
    "aa-shadow_dataset-align-top_down_weight_matching",
]

In [3]:
roc_results = {attacker_access: 
               {f : defaultdict(list) 
                for f in ['activations', 'gradients', 'activations,gradients']} 
               for attacker_access in attacker_accesses}

In [4]:
experiments_dir = '../experiments/tiny-imagenet-200/attack/resnet18/attack_results'
num_repetitions = 5

for features in ['activations', 'gradients']:
    print(f'\nFeatures: {features} \n')

    if features == "activations":
        target_layers = ["fc1", "fc1-ia", "fc1-ia-only"]
    elif features == 'gradients':
        target_layers = ["fc1", "fc1-ia"]

    for attacker_access in attacker_accesses:
        print(f'\nAttacker access: {attacker_access}')
        results = {layer: {"test_acc": [], "test_auc": [], "best_test_acc": []} for layer in target_layers}

        for layer in target_layers:
            num_repetitions_found = 0
            for exp in range(num_repetitions):
                result_dir = os.path.join(experiments_dir, attacker_access, features, layer)
                saved_model_path = os.path.join(result_dir, f'exp_{exp}_model.pickle')
                if not os.path.exists(saved_model_path):
                    continue
                with open(saved_model_path, 'rb') as f:
                    saved_model = pickle.load(f)
                    if not saved_model['train_complete']:
                        continue
                    test_metrics = saved_model['test_metrics']
                test_acc = test_metrics['acc']
                test_auc = test_metrics['auc']
                results[layer]['test_auc'].append(test_auc)
                results[layer]['test_acc'].append(test_acc)
                roc_results[attacker_access][features][layer].append( 
                    (test_metrics['fpr'], test_metrics['tpr']) )
                num_repetitions_found += 1
            if num_repetitions_found == 0:
                continue
            mean_test_auc, std_test_auc = np.mean(results[layer]['test_auc']), np.std(results[layer]['test_auc'])
            mean_test_acc, std_test_acc = np.mean(results[layer]['test_acc']), np.std(results[layer]['test_acc'])
            h_test_auc =  std_test_auc * scipy.stats.t.ppf((1 + 0.95) / 2., num_repetitions_found-1) / (num_repetitions_found**0.5)
            print(f'Layer {layer}: {num_repetitions_found} experiments. ', 
                f'Test auc {mean_test_auc:.3f} ({h_test_auc:.3f}), test acc: {mean_test_acc:.1%} ({std_test_acc:.1%})')


Features: activations 


Attacker access: aa-target_dataset
Layer fc1: 5 experiments.  Test auc 0.597 (0.010), test acc: 56.8% (0.7%)
Layer fc1-ia: 5 experiments.  Test auc 0.687 (0.010), test acc: 64.3% (0.6%)
Layer fc1-ia-only: 5 experiments.  Test auc 0.688 (0.012), test acc: 63.9% (0.8%)

Attacker access: aa-shadow_dataset_model_init
Layer fc1: 5 experiments.  Test auc 0.586 (0.002), test acc: 55.5% (0.7%)
Layer fc1-ia: 5 experiments.  Test auc 0.681 (0.010), test acc: 63.3% (0.4%)
Layer fc1-ia-only: 5 experiments.  Test auc 0.682 (0.010), test acc: 63.4% (0.4%)

Attacker access: aa-shadow_dataset
Layer fc1: 5 experiments.  Test auc 0.586 (0.012), test acc: 55.8% (0.5%)
Layer fc1-ia: 5 experiments.  Test auc 0.667 (0.010), test acc: 61.5% (1.0%)
Layer fc1-ia-only: 5 experiments.  Test auc 0.667 (0.002), test acc: 62.5% (0.8%)

Attacker access: aa-shadow_dataset-align-top_down_weight_matching
Layer fc1-ia-only: 5 experiments.  Test auc 0.675 (0.009), test acc: 62.5% (0.3%)

Feature

In [5]:
features = 'activations,gradients'
print(f'\nFeatures: {features} \n')

target_layers = ["fc1", "fc1-ia"]

for attacker_access in attacker_accesses:
    print(f'\nAttacker access: {attacker_access}')
    results = {layer: {"test_acc": [], "test_auc": [], "best_test_acc": []} for layer in target_layers}

    for layer in target_layers:
        num_repetitions_found = 0
        for exp in range(num_repetitions):
            result_dir = os.path.join(experiments_dir, attacker_access, features, layer)
            saved_model_path = os.path.join(result_dir, f'exp_{exp}_model.pickle')
            if not os.path.exists(saved_model_path):
                continue
            with open(saved_model_path, 'rb') as f:
                saved_model = pickle.load(f)
                if not saved_model['train_complete']:
                    continue
                test_metrics = saved_model['test_metrics']
            test_acc = test_metrics['best_acc']
            test_auc = test_metrics['auc']
            results[layer]['test_auc'].append(test_auc)
            results[layer]['test_acc'].append(test_acc)
            num_repetitions_found += 1
            roc_results[attacker_access][features][layer].append( 
                    (test_metrics['fpr'], test_metrics['tpr']) )
        if num_repetitions_found == 0:
            continue
        mean_test_auc, std_test_auc = np.mean(results[layer]['test_auc']), np.std(results[layer]['test_auc'])
        mean_test_acc, std_test_acc = np.mean(results[layer]['test_acc']), np.std(results[layer]['test_acc'])
        h_test_auc =  std_test_auc * scipy.stats.t.ppf((1 + 0.95) / 2., num_repetitions_found-1) / (num_repetitions_found**0.5)
        print(f'Layer {layer}: {num_repetitions_found} experiments. ', 
            f'Test auc {mean_test_auc:.3f} ({h_test_auc:.3f}), best test acc: {mean_test_acc:.1%} ({std_test_acc:.1%})')


Features: activations,gradients 


Attacker access: aa-target_dataset
Layer fc1: 5 experiments.  Test auc 0.685 (0.005), best test acc: 64.2% (0.1%)
Layer fc1-ia: 5 experiments.  Test auc 0.693 (0.008), best test acc: 64.7% (0.3%)

Attacker access: aa-shadow_dataset_model_init
Layer fc1: 5 experiments.  Test auc 0.681 (0.004), best test acc: 63.9% (0.8%)
Layer fc1-ia: 5 experiments.  Test auc 0.694 (0.005), best test acc: 64.4% (0.9%)

Attacker access: aa-shadow_dataset
Layer fc1: 5 experiments.  Test auc 0.680 (0.003), best test acc: 63.5% (0.5%)
Layer fc1-ia: 5 experiments.  Test auc 0.689 (0.007), best test acc: 64.8% (0.4%)

Attacker access: aa-shadow_dataset-align-top_down_weight_matching
Layer fc1: 5 experiments.  Test auc 0.684 (0.004), best test acc: 64.1% (0.6%)
Layer fc1-ia: 5 experiments.  Test auc 0.691 (0.008), best test acc: 64.8% (0.5%)
